In [16]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

# Path to input and output video files
input_video_path = 'xtdowner.com_127498_mp4_360P.mp4'  # Replace with your input video path
output_video_path = 'output_edge_video.mp4'

# Set up video capture and properties
cap = cv2.VideoCapture(input_video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Set up video writer to save output
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height), isColor=False)

# Process each frame
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Resize the frame for moderate detail
    scale_factor = 0.8
    frame = cv2.resize(frame, (int(frame.shape[1] * scale_factor), int(frame.shape[0] * scale_factor)))

    # Apply contrast enhancement
    alpha = 1.5  # Increased contrast
    beta = 50    # Slight brightness increase
    contrast_frame = cv2.convertScaleAbs(frame, alpha=alpha, beta=beta)

    # Split the contrast-enhanced frame into Red, Green, and Blue channels
    blue_channel, green_channel, red_channel = cv2.split(contrast_frame)

    # Apply selective Canny edge detection on each channel
    edges_red = cv2.Canny(red_channel, threshold1=100, threshold2=250)
    edges_green = cv2.Canny(green_channel, threshold1=100, threshold2=250)
    edges_blue = cv2.Canny(blue_channel, threshold1=50, threshold2=200)  # More sensitive to blue

    # Combine edges from each channel with emphasis on blue
    edges_combined = cv2.bitwise_or(edges_blue, cv2.bitwise_or(edges_red, edges_green))

    # Resize edges_combined to match the original frame dimensions for consistent output
    edges_combined = cv2.resize(edges_combined, (frame_width, frame_height))

    # Write the processed frame to the output video
    out.write(edges_combined)

# Release resources
cap.release()
out.release()
print("Edge-detected video with selective settings has been saved as", output_video_path)

Edge-detected video with selective settings has been saved as output_edge_video.mp4
